# Optimización de Recursos Humanos:  
# Predicción y Gestión de Horas Extras con Modelos de Machine Learning y Series Temporales


Universidad Internacional de La Rioja  
Escuela Superior de Ingeniería y Tecnologa  
Máster Universitario en Análisis y Visualización de Datos Masivos/ Visual Analytics and Big Data


**Por Juliana Betancur Morales**

## Anonimización de datos

Como parte fundamental del proceso, se implementará la anonimización de los datos personales de los empleados de la compañía, en cumplimiento de la Ley 1581 de 2012 sobre Protección de Datos Personales. Este proceso de anonimización se aplicará tanto a los datos correspondientes al periodo 2013-2022 como a los registros de 2022 a 2024. Sin embargo, mientras que los datos de 2013 a 2022 serán sometidos a un proceso de preprocesamiento para su adecuada estructuración y limpieza antes del análisis, los datos de 2022 a 2024 únicamente serán anonimizados sin realizar ninguna transformación adicional.

### **Importación de librerias necesarias**

En el presente estudio, se lleva a cabo el procesamiento y tratamiento de los datos extraídos del sistema de nómina utilizado hasta el año 2022. El análisis abarca un periodo histórico desde el año 2013, permitiendo así una evaluación integral y longitudinal de la información.

In [3]:
import pandas as pd
import hashlib
import os
import re
import numpy as np

### **Preprocesamiento de datos y anonimización datos 2013-2021**

In [4]:
# Carpetas de entrada y salida
input_folder = "Historico"
output_folder = "Historico_Anon"
os.makedirs(output_folder, exist_ok=True)

# Definimos nuevos encabezados
new_headers = [
    "Inicial", "Empleado_ID", "Nombre", "Tipo", "Tipo_Empleado", "Fecha", "Fecha_Vinculacion",
    "Sec_Vinculacion", "num_sec", "Tipo", "Tipo_Nomina", "Sec_Detalle", "Num_sec_deta", "Centro", "Centro_Costos", "Oficio", "Num_Oficio",
    "Sec_Plaza", "Num_Plaza", "Sal", "Salario", "Concepto_1", "Horas_1", "Tipo_Mvto_1", "Valor_1",
    "Concepto_2", "Horas_2", "Tipo_Mvto_2", "Valor_2", "Valor_Total", "Total_Horas", "Tipo de concepto", "Signo", "Concepto", "Total_empleado",
    "ID", "Devengados", "Valor_dev", "Deducciones", "Valor_ded", "Neto", "Valor_neto"
]

# Limpiar y unificar el formato de los datos en el archivo
def clean_data(content):
    # Eliminar las comas dentro de los números (sin afectar las comas entre columnas)
    content = re.sub(r'(?<=\d),(?=\d)', '', content)
    return content

# Limpiar valores numéricos (remover caracteres no deseados)
def clean_value(value):
    try:
        return float(re.sub(r'[^0-9.]', '', str(value)))
    except:
        return np.nan

# Función para anonimizar los valores de las columnas
def anonymize_column(value):
    if pd.isna(value):
        return value
    # Usar hashlib para crear un hash de la columna 
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

# Procesar un archivo TXT en un DataFrame
def process_txt_to_df(file_path):
    try:
        # Leer el archivo con codificación ISO-8859-1
        with open(file_path, 'r', encoding='ISO-8859-1') as f:
            content = f.read().strip()
        
        if not content:
            print(f"El archivo {file_path} está vacío.")
            return None

        # Limpiar los datos del archivo
        content = clean_data(content)

        # Dividir el contenido en líneas
        lines = content.splitlines()
        
        # Crear una lista para almacenar los registros procesados
        records = []
        
        for line in lines:
            # Eliminar filas no deseadas
            if "TOTALES POR CONCEPTOS" in line or "Valor total" in line:
                continue

            # Dividir cada línea en columnas según la coma
            data = line.split(',')
            
            # Mapear los datos a los nuevos encabezados
            formatted_data = {header: (data[i] if i < len(data) else None) for i, header in enumerate(new_headers)}
            records.append(formatted_data)
        
        # Crear un DataFrame con todos los registros procesados
        df = pd.DataFrame(records)
        
        # Anonimizar las columnas 'Empleado_ID' y 'Nombre'
        df['Empleado_ID'] = df['Empleado_ID'].apply(anonymize_column)
        df['Nombre'] = df['Nombre'].apply(anonymize_column)

        # Limpiar los valores numéricos en las columnas relevantes
        df['Salario'] = df['Salario'].apply(clean_value)
        df['Valor_Total'] = df['Valor_Total'].apply(clean_value)

        # Extraer la parte entera de la columna Total_Horas
        df['Total_Horas'] = pd.to_numeric(df['Total_Horas'], errors='coerce').apply(np.floor).astype('float')

        return df

    except Exception as e:
        print(f"Error procesando el archivo {file_path}: {e}")
        return None

# Procesar todos los archivos TXT en la carpeta de entrada
def main():
    # Obtener la lista de archivos en la carpeta de entrada
    txt_files = [file for file in os.listdir(input_folder) if file.lower().endswith('.txt')]
    
    if not txt_files:
        print("No se encontraron archivos TXT en la carpeta de entrada.")
        return
    
    for txt_file in txt_files:
        file_path = os.path.join(input_folder, txt_file)
        print(f"Procesando archivo: {file_path}")
        
        df = process_txt_to_df(file_path)
        
        if df is not None:
           
            
            # Guardar el archivo procesado como archivo TXT
            output_file = os.path.join(output_folder, f"{os.path.splitext(txt_file)[0]}.txt")
            
            # Guardar el DataFrame como un archivo .txt con comas como separadores
            df.to_csv(output_file, index=False, sep=',', encoding='')
            print(f"Archivo TXT guardado en: {output_file}")
        else:
            print(f"No se pudo procesar el archivo: {file_path}")

# Ejecutar la función principal
if __name__ == "__main__":
    main()


Procesando archivo: Historico\Abril 2013.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2013.txt
Procesando archivo: Historico\Abril 2014.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2014.txt
Procesando archivo: Historico\Abril 2015.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2015.txt
Procesando archivo: Historico\Abril 2016.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2016.txt
Procesando archivo: Historico\Abril 2017.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2017.txt
Procesando archivo: Historico\Abril 2018.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2018.txt
Procesando archivo: Historico\Abril 2019.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2019.txt
Procesando archivo: Historico\Abril 2020.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2020.txt
Procesando archivo: Historico\Abril 2021.TXT
Archivo TXT guardado en: Historico_Anon\Abril 2021.txt
Procesando archivo: Historico\Agosto 2013.TXT
Archivo TXT guardado en: Historico_Anon\Agosto 2013.tx

### **Anonimización datos 2022-2024**

In [5]:
# Carpetas de entrada y salida
input_folder = "Cambio_de_sistema"
output_folder = "Cambio_de_sistema_Anon"

# Crea la carpeta de salida en caso de que no exista
os.makedirs(output_folder, exist_ok=True)

# Función para anonimizar los valores de una columna
def anonymize_column(value):
    if pd.isna(value):  # Manejo de valores nulos
        return value
    # Crear un hash de la columna usando SHA-256
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

# Función principal para procesar y anonimizar archivos
def process_and_anonymize_files():
    # Obtener la lista de archivos .xlsx en la carpeta de entrada
    files = [f for f in os.listdir(input_folder) if f.lower().endswith('.xlsx')]
    
    if not files:
        print("No se encontraron archivos .xlsx en la carpeta de entrada.")
        return

    for file_name in files:
        input_file_path = os.path.join(input_folder, file_name)
        print(f"Procesando archivo: {file_name}")
        
        try:
            # Leer el archivo Excel
            df = pd.read_excel(input_file_path)
            
            # Anonimizar las columnas
            if 'Identificación Empleado' in df.columns:
                df['Identificación Empleado'] = df['Identificación Empleado'].apply(anonymize_column)
            if 'Empleado' in df.columns:
                df['Empleado'] = df['Empleado'].apply(anonymize_column)
            
            # Crear la ruta del archivo de salida
            output_file_name = os.path.splitext(file_name)[0] + "_Anonimizado.xlsx"
            output_file_path = os.path.join(output_folder, output_file_name)
            
            # Guardar el DataFrame anoninimizado como archivo Excel
            df.to_excel(output_file_path, index=False, engine='openpyxl')
            print(f"Archivo anonimizado guardado en: {output_file_path}")
        
        except Exception as e:
            print(f"Error procesando el archivo {file_name}: {e}")

# Ejecutar la función principal
if __name__ == "__main__":
    process_and_anonymize_files()


Procesando archivo: 2022.xlsx
Archivo anonimizado guardado en: Cambio_de_sistema_Anon\2022_Anonimizado.xlsx
Procesando archivo: 2023.xlsx
Archivo anonimizado guardado en: Cambio_de_sistema_Anon\2023_Anonimizado.xlsx
Procesando archivo: 2024.xlsx
Archivo anonimizado guardado en: Cambio_de_sistema_Anon\2024_Anonimizado.xlsx
